In [1]:
import pandas
import geopandas 
import feedparser
import requests
from   zipfile import ZipFile
import datetime 
from   local_module import NhcDownloaderBot

%matplotlib inline

In [2]:
url = 'https://www.nhc.noaa.gov/index-at.xml'
f = feedparser.parse(url)
f

{'bozo': False,
 'entries': [{'title': 'Atlantic Tropical Weather Outlook',
   'title_detail': {'type': 'text/plain',
    'language': None,
    'base': 'https://www.nhc.noaa.gov/index-at.xml',
    'value': 'Atlantic Tropical Weather Outlook'},
   'summary': '<br />\n000<br />\nABNT20 KNHC 201735<br />\nTWOAT <br />\n<br />\nTropical Weather Outlook<br />\nNWS National Hurricane Center Miami FL<br />\n200 PM EDT Mon Sep 20 2021<br />\n<br />\nFor the North Atlantic...Caribbean Sea and the Gulf of Mexico:<br />\n<br />\nThe National Hurricane Center is issuing advisories on Tropical <br />\nStorm Peter, located northeast of the northernmost Leeward <br />\nIslands, and on Tropical Storm Rose, located over the eastern <br />\ntropical Atlantic Ocean.<br />\n<br />\nShowers and thunderstorms associated with a tropical wave located<br />\nseveral hundred miles south of the Cabo Verde Islands are <br />\nstill showing some signs of organization, although there does not <br />\nappear to be a

In [3]:
#f
#f.keys()
# len(f.entries)
# f['entries'][1]['nhc_atcf']
# f.status
# f.feed
# f.feed['subtitle']
# f.entries
# type(f.entries)
# for entry in f.entries:
#     print(entry.title)

In [3]:
df = pandas.DataFrame(f.entries)
df = df.drop(columns=['title_detail', 'summary', 'summary_detail', 'published_parsed', 'links', 'link', 'id', 
                 'guidislink', 'authors', 'author', 'author_detail'])
df.head()

,title,published,nhc_center,nhc_type,nhc_name,nhc_wallet,nhc_atcf,nhc_datetime,nhc_movement,nhc_pressure,nhc_wind,nhc_headline,nhc_cyclone
0,Atlantic Tropical Weather Outlook,"Mon, 20 Sep 2021 17:35:32 GMT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Summary for Tropical Storm Peter (AT1/AL162021),"Mon, 20 Sep 2021 14:44:23 GMT","19.5, -60.9",TROPICAL STORM,Peter,AT1,AL162021,11:00 AM AST Mon Sep 20,WNW at 15 mph,1007 mb,50 mph,...PETER BATTLING INCREASINGLY HOSTILE WIND SH...,
2,Tropical Storm Peter Public Advisory Number 8,"Mon, 20 Sep 2021 14:44:23 GMT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Tropical Storm Peter Forecast Advisory Number 8,"Mon, 20 Sep 2021 14:44:23 GMT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Tropical Storm Peter Forecast Discussion Number 8,"Mon, 20 Sep 2021 14:44:53 GMT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
tc_list = [nhc_atcf[2:4] for nhc_atcf in df['nhc_atcf'] if pandas.isnull(nhc_atcf) == False] 
tc_list

['16', '17']

In [5]:
#{key: value for (key, value) in iterable}
d = {nhc_name: [nhc_atcf    , nhc_type, nhc_center, nhc_movement, 
                nhc_pressure, nhc_wind, published , nhc_datetime]         
     for (nhc_name    , nhc_atcf, nhc_type  , nhc_center  , nhc_movement,
          nhc_pressure, nhc_wind, published , nhc_datetime) 
     in  zip(df['nhc_name']    , df['nhc_atcf']    , df['nhc_type'], df['nhc_center'], 
             df['nhc_movement'], df['nhc_pressure'], df['nhc_wind'], df['published'],
             df['nhc_datetime']) 
     if pandas.isnull(nhc_name) == False}   
d

{'Peter': ['AL162021',
  'TROPICAL STORM',
  '19.5, -60.9',
  'WNW at 15 mph',
  '1007 mb',
  '50 mph',
  'Mon, 20 Sep 2021 14:44:23 GMT',
  '11:00 AM AST Mon Sep 20'],
 'Rose': ['AL172021',
  'Tropical Storm',
  '17.3, -33.4',
  'NW at 16 mph',
  '1007 mb',
  '40 mph',
  'Mon, 20 Sep 2021 14:43:55 GMT',
  '11:00 AM AST Mon Sep 20']}

In [6]:
class NhcDownloaderBot:
    def __init__(self, storm_number, year):
        self.storm_number = storm_number
        self.year         = year
        
        
        self.file_names     = file_names = [
                            f'al{self.storm_number}{self.year}_5day_latest.zip',
                            f'al{self.storm_number}{self.year}_fcst_latest.zip',
                            f'al{self.storm_number}{self.year}_best_track.zip',
                            f'gtwo_shapefiles.zip',
                            f'wsp_120hrhalfDeg_latest.zip',
                            f'wsp_120hr5km_latest.zip'
                            ]
        self.urls           = urls = [
                            f'https://www.nhc.noaa.gov/gis/forecast/archive',
                            f'https://www.nhc.noaa.gov/gis/forecast/archive',
                            f'https://www.nhc.noaa.gov/gis/best_track',
                            f'https://www.nhc.noaa.gov/xgtwo',
                            f'https://www.nhc.noaa.gov/gis/forecast/archive',
                            f'https://www.nhc.noaa.gov/gis/forecast/archive'
                            ]
        self.gdf_names      = gdf_names = {
                            file_names[0] : ['track_line_gdf', 'cone_gdf', 'points_gdf'],
                            file_names[1] : ['init_radii_gdf', 'fcst_radii_gdf'],
                            file_names[2] : ['best_track_points_gdf', 'best_track_line_gdf', 'best_track_radii_gdf', 'best_track_swath_gdf'],
                            file_names[3] : ['gtwo_areas_gdf', 'gtwo_lines_gdf', 'gtwo_points_gdf'],
                            file_names[4] : ['wsp_34_gdf_points', 'wsp_50_gdf_points', 'wsp_64_gdf_points'],
                            file_names[5] : ['wsp_34_gdf_polygons', 'wsp_50_gdf_polygons', 'wsp_64_gdf_polygons']
                                          }
    def nhc_gis_downloader(self, file_names, urls, gdf_names):

        for idx, file_name in enumerate(file_names):
            url = f'{urls[idx]}/{file_name}'
            r = requests.get(url)
            with open(f'nhc_latest/{file_name}', 'wb') as code:
                code.write(r.content) 
        _  = [0, 3, 4, 5]  
        __ = [2, 7, 12]  
        for idx, file_name in enumerate([file_names[0],  file_names[3], 
                                        file_names[4],  file_names[5]]): 
            with ZipFile(f'nhc_latest/{file_name}', 'r') as zip_file_name:
                for idx2 in range(len(gdf_names[file_names[_[idx]]])):
                    gdf_names[file_names[_[idx]]][idx2]  = geopandas.read_file(
                    f'zip://./nhc_latest/{file_name}!{zip_file_name.namelist()[__[idx2]]}')
        _  = [0,5]                                 
        with ZipFile(f'nhc_latest/{file_names[1]}', 'r') as zip_file_name:
                for idx in range(2):
                    gdf_names[file_names[1]][idx]  = geopandas.read_file(
                    f'zip://./nhc_latest/{file_names[1]}!{zip_file_name.namelist()[_[idx]]}')
        _  = [2, 7, 13, 16]                                 
        with ZipFile(f'nhc_latest/{file_names[2]}', 'r') as zip_file_name:
            for idx in range(4):
                gdf_names[file_names[2]][idx]  = geopandas.read_file(
                f'zip://./nhc_latest/{file_names[2]}!{zip_file_name.namelist()[_[idx]]}')
        return gdf_names
    
    @classmethod
    def gdf_file(cls, storm_number, year):
        cls._                     = NhcDownloaderBot(storm_number, year)
        cls.track_line_gdf        = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[0]][0]
        cls.cone_gdf              = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[0]][1]
        cls.points_gdf            = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[0]][2]
        cls.init_radii_gdf        = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[1]][0]
        cls.fcst_radii_gdf        = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[1]][1]
        cls.best_track_points_gdf = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[2]][0]
        cls.best_track_line_gdf   = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[2]][1]
        cls.best_track_radii_gdf  = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[2]][2]
        cls.best_track_swath_gdf  = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[2]][3]
        cls.gtwo_areas_gdf        = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[3]][0]
        cls.gtwo_lines_gdf        = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[3]][1]
        cls.gtwo_points_gdf       = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[3]][2]
        cls.wsp_34_gdf_points     = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[4]][0]
        cls.wsp_50_gdf_points     = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[4]][1]
        cls.wsp_64_gdf_points     = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[4]][2]
        cls.wsp_34_gdf_polygons   = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[5]][0]
        cls.wsp_50_gdf_polygons   = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[5]][1]
        cls.wsp_64_gdf_polygons   = cls._.nhc_gis_downloader(cls._.file_names, cls._.urls, cls._.gdf_names)[cls._.file_names[5]][2]

In [7]:
NhcDownloaderBot.gdf_file(16, 2021)
print('Done')

Done


In [11]:
NhcDownloaderBot.track_line_gdf

,STORMNAME,STORMTYPE,ADVDATE,ADVISNUM,STORMNUM,FCSTPRD,BASIN,geometry
0,Larry,PTC,1100 AM AST Sat Sep 11 2021,44,12.0,120.0,AL,"LINESTRING (-48.20000 54.00000, -44.90000 57.8..."


In [9]:
ct = []
for tc in tc_list:
    df = NhcDownloaderBot(tc, 2021)
    df = df.gdf_file(tc, 2021)
    ct.append(NhcDownloaderBot.track_line_gdf)
    if tc == tc_list[-1]:
        print(f'--------------------done-----------------------')
    
#     NhcDownloaderBot.init_radii_gdf
    

--------------------done-----------------------


In [12]:
ct[0]

,STORMNAME,STORMTYPE,ADVDATE,ADVISNUM,STORMNUM,FCSTPRD,BASIN,geometry
0,Peter,TS,1100 AM AST Mon Sep 20 2021,8,16.0,120.0,AL,"LINESTRING (-60.90000 19.50000, -62.70000 20.1..."
